In [1]:
from mortgage_calculator import MortgageCalculator

# Mortgage info and calculations
home_cost = 500_000
loan_term = 30
interest_rate = 0.05
down_payment_percent = 0.2
mortgage_calc = MortgageCalculator(home_cost, loan_term, interest_rate, down_payment_percent)

In [3]:
print(f"Home cost: ${home_cost} Down Payment ${mortgage_calc.down_payment_amount:.2f} Loan ${mortgage_calc.loan_amount}")
print(f"Monthly payment $ + {mortgage_calc.monthly_payment:.2f}")

Home cost: $500000 Down Payment $100000.00 Loan $400000.0
Monthly payment $ + 2147.29


In [5]:
from tax_calculator import FilingStatus, FederalTaxCalculator, StateTaxCalculator

# Income tax info
filing_status = FilingStatus.SINGLE
gross_income = 100_000
# Deductions taken to calculate gross income (ex. 401k, Health insurance premiums FSA, HSA, Traditional IRA, etc.)
above_line_deductions = 0
# Deductions taken to calculate taxable income, might not apply if less than standard deduction (ex. some medical expenses, charitable donations)
# Exclude mortgage interest and state and local taxes for now, we'll calculate these and account for them
itemized_deductions = 0

# Taxes before mortgage
ny_tax_calc = StateTaxCalculator("NY", filing_status, gross_income, above_line_deductions, itemized_deductions)
state_income_tax_wo_mortgage, marginal_rate = ny_tax_calc.calc_income_taxes_without_mortgage()

fed_tax_calc = FederalTaxCalculator(filing_status, gross_income, above_line_deductions, itemized_deductions)
# TODO include state income tax
fed_taxes_wo_mortgage, marg_rate_wo = fed_tax_calc.calc_taxes_without_mortgage(state_income_tax_wo_mortgage)
print(f"Taxes before mortgage and property taxes: Federal Income Tax ${fed_taxes_wo_mortgage:.2f}, State Income Tax ${state_income_tax_wo_mortgage:.2f}")    

# Taxes after mortgage, accounts for interest paid in the first year. Subsequent years will have lower interest payment and therefore tax savings
property_tax = ny_tax_calc.calc_property_tax(home_cost)
year_one_interest = mortgage_calc.get_year_one_interest()
state_income_tax_w_mortgage, _ = ny_tax_calc.calc_income_taxes_with_mortgage(mortgage_calc.loan_amount, year_one_interest)
total_state_tax = property_tax + state_income_tax_wo_mortgage

fed_taxes_w_mortgage, marg_rate_w = fed_tax_calc.calc_taxes_with_mortgage(mortgage_calc.loan_amount, year_one_interest, total_state_tax)
print(f"Taxes after mortgage and property taxes: Federial Income Tax ${fed_taxes_w_mortgage:.2f}, State Income Tax ${state_income_tax_w_mortgage:.2f}, Property Tax ${property_tax:.2f}")

total_taxes_wo_mortgage = fed_taxes_wo_mortgage + state_income_tax_wo_mortgage
total_taxes_w_mortgage = fed_taxes_w_mortgage + state_income_tax_w_mortgage + property_tax
additional_taxes = total_taxes_w_mortgage - total_taxes_wo_mortgage
print(f"Total taxes before mortgage: ${total_taxes_wo_mortgage:.2f}, Total taxes after mortgage: ${total_taxes_w_mortgage:.2f}, Additional Taxes: ${additional_taxes:.2f}")

income_tax_savings = (state_income_tax_wo_mortgage - state_income_tax_w_mortgage) + (fed_taxes_wo_mortgage - fed_taxes_w_mortgage)
print(f"Income tax savings: ${income_tax_savings:.2f}")

Taxes before mortgage and property taxes: Federal Income Tax $13841.00, State Income Tax $4951.75
Taxes after mortgage and property taxes: Federial Income Tax $12522.06, State Income Tax $4796.04, Property Tax $7000.00
Total taxes before mortgage: $18792.75, Total taxes after mortgage: $24318.10, Additional Taxes: $5525.35
Income tax savings: $1474.65


In [27]:
# Year one home owernship costs. Note: does not include insurance, HOA, maintenance, etc.
mortgage_payments = mortgage_calc.annual_payments
total_annual_cost = mortgage_payments + property_tax - income_tax_savings
monthly_cost = total_annual_cost / 12
print(f"Total Annual Cost: ${total_annual_cost:.2f}, Monthly Cost: ${monthly_cost:.2f}")

Total Annual Cost: $41795.47, Monthly Cost: $3482.96
